In [6]:
from chain import get_sentence
import cPickle as pickle
import tweepy
from datetime import date, timedelta
#keys = PATH(../keys)
from ../keys import keys

SyntaxError: invalid syntax (<ipython-input-6-2c030eb06aa4>, line 6)

In [3]:
with open('Pickles/forward_dict.pkl') as f:
    f_dict = pickle.load(f)

with open('Pickles/reversed_dict.pkl') as f:
    r_dict = pickle.load(f)

NameError: name 'pickle' is not defined

In [30]:
CONSUMER_KEY = keys['consumer_key']
CONSUMER_SECRET = keys['consumer_secret']
ACCESS_TOKEN = keys['access_token']
ACCESS_TOKEN_SECRET = keys['access_token_secret']

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

tweet_replies = []

today = date.today().strftime("%Y-%m-%d")
yesterday = date.today() - timedelta(1)

# ******************************************************
# Queries Popular Tweets directed at Candidates or using
# well-known Hashtags and responds to all queried tweets.
# ******************************************************
hashtags = ["#FeelTheBern OR #ImWithHer OR #MakeAmericaGreatAgain OR #Trump2016 \
OR @BernieSanders OR @HillaryClinton OR @realDonaldTrump"]

top_tweets = tweepy.Cursor(api.search, q=hashtags, count=100, result_type='popular', include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, lang="en", since=yesterday, until=today).items()

for status in top_tweets:
    if status.favorite_count > 1000:
        tweet_replies.append(status)

# ***************************************************
# Queries Tweets directed at or in reply to
# FeelTheBern_Bot and responds to all queried tweets.
# ***************************************************
bernbot_query = "to:FeelTheBern_Bot OR @FeelTheBern_Bot"

bernbot_tweets = tweepy.Cursor(api.search, q=bernbot_query, count=100, result_type='recent', include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, lang="en", since=yesterday, until=today).items()

for status in bernbot_tweets:
    tweet_replies.append(status)

# ************************************************
# Queries Tweets from Presidential Candidates and
# responds to those with greater than 500 likes.
# ************************************************
candidate_query = "from:BernieSanders OR from:HillaryClinton OR from:realDonaldTrump"

candidate_tweets = tweepy.Cursor(api.search, q=candidate_query, count=100, result_type='recent', include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, lang="en", since=yesterday, until=today).items()

for status in candidate_tweets:
    if status.favorite_count > 500:
        tweet_replies.append(status)


# DROPS ANY DUPLICATE TWEETS SO IT DOES NOT RESPOND TWICE
tweet_replies = set(tweet_replies)

In [31]:
best_tweets = []
for i, tweet in enumerate(tweet_replies):
    resp_length = 141
    text = tweet.text
    sn = tweet.user.screen_name
    response = ':'

    while resp_length > 140 or ':' in response:
        response = get_sentence(text, f_dict, r_dict, randomness = 1) 
        response = "@{0} {1}".format(sn, response)
        resp_length = len(response)
    
    print i, '----', tweet.id
    print '@', sn, text
    print response
    print '*************'
    best_tweets.append([response, tweet.id])

0 ---- 727141602917937155
@ BernieSanders I want to make banking boring again. It shouldn't be about making as much profit as possible by gambling on esoteric financial products.
@BernieSanders She has now promised here tonight not to raise them on families Making $250 ,000 or less.
*************
1 ---- 727114876485693440
@ DanScavino United States Air Force pilots greet @realDonaldTrump in Indianapolis, Indiana. #INPrimary #Trump2016 #BTS #MAGA https://t.co/V01l9hbFgx
@DanScavino States of America is at a gun show, then we go in with specials ops followed by our air force to take them over.
*************
2 ---- 726937251461140482
@ HillaryClinton "Systemic racism, inter-generational poverty, gun violence—these aren’t somebody else’s problems. These are our problems." —Hillary
@HillaryClinton He was the chairman of the Congressional Black Caucus endorsed Secretary Clinton, can we stick to gun control?
*************
3 ---- 727169840776118272
@ realDonaldTrump I will defeat Crooked Hilla

In [29]:
best_list = [0,1,7,8,14,17,24,39]
for i, tweet in enumerate(best_tweets):
    if i in best_list:
        print i 
        print tweet[0], tweet[1]
        s = api.update_status(tweet[0], tweet[1])

0
@HillaryClinton At a town hall meeting, and she cannot tell the American people the courage to stand up to do it. 726938033493299200
1
@HillaryClinton Trump has, as you are well aware, denounced that statement over and over. 727280763381522432
7
@HillaryClinton I've done it once in Washington, with great jobs and lower taxes. 726931949038751744
8
@DanScavino The air strikes are important, but we need to stand up against the gun lobby. 727114876485693440
14
@HillaryClinton Black and white, Latino, Asian-American. 726933701540610048
17
@realDonaldTrump You don't make America great by getting rid of everything that made America great. 727225626575421440
24
@realDonaldTrump As Secretary Clinton may know, I enjoy the interplay. 727085972278677504
39
@HillaryClinton We're closing hospitals, we're closing wards, we're closing so many because the states want to save our country. 726962211290120192
